In [1]:
# DO _ DOID translator


In [2]:
# 라이브러리 호출
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
# 데이터 불러오기
rawData = pd.read_csv('TPSS_region_OD3.csv', encoding='cp949', sep=",")
rawData.head()

,Date,OA,OB,DA,DB,population
0,20230920,성동구,성수2가1동,은평구,대조동,3
1,20230920,성동구,성수2가1동,은평구,불광1동,2
2,20230920,성동구,성수2가1동,은평구,불광2동,1
3,20230920,성동구,성수2가1동,은평구,수색동,2
4,20230920,성동구,성수2가1동,은평구,신사1동,3


In [4]:
filterdDf= rawData.loc[:,['OA','OB','DA','DB']]


In [5]:

# A에서 B로 이동하는 데이터 복사
output1 = filterdDf.copy()

# B에서 A로 이동하는 데이터 복사
output2 = filterdDf.copy()

# 열 이름 바꾸기
output2.columns = ['DA', 'DB','OA','OB' ]


# 두 데이터프레임 병합
merged_df = pd.concat([output1, output2], axis=0)


# (Location_1=지역1 & Location_2=지역3)와 (Location_1=지역3 & Location_2=지역1)을 같은 것으로 처리
merged_df['Unique_DB'] = merged_df[['DB', 'OB']].min(axis=1)
merged_df['Unique_OB'] = merged_df[['DB', 'OB']].max(axis=1)

merged_df['Unique_DA'] = merged_df[['DA', 'OA']].min(axis=1)
merged_df['Unique_OA'] = merged_df[['DA', 'OA']].max(axis=1)

# 중복된 데이터 제거
merged_df = merged_df.drop_duplicates(subset=['Unique_DA','Unique_DB'])

# 불필요한 열 제거
merged_df = merged_df.drop(columns=['Unique_DA','Unique_DB', 'Unique_OA','Unique_OB','DA','DB'])

#중복제거
merged_df = merged_df.drop_duplicates(subset=['OA','OB'])




In [6]:
merged_df.head(300)

,OA,OB
0,성동구,성수2가1동
56,성동구,성수2가3동
144,성동구,왕십리도선동
280,성북구,길음1동
433,성북구,삼선동
...,...,...
88710,중랑구,상봉1동
88920,평택시,세교동
89092,화성시,진안동
89893,성동구,금호4가동


In [7]:
merged_df['OB'].unique()

array(['성수2가1동', '성수2가3동', '왕십리도선동', '길음1동', '삼선동', '월곡1동', '정릉1동',
       '정릉2동', '가락1동', '마천2동', '문정1동', '석촌동', '위례동', '잠실6동', '잠실7동',
       '매탄3동', '신현동', '연성동', '일동', '갈산동', '관양1동', '석수1동', '석수2동', '회천1동',
       '회천2동', '목1동', '목2동', '목4동', '신월1동', '신월4동', '신월5동', '신정1동',
       '신정4동', '옥천면', '용문면', '옥련2동', '당산1동', '대림1동', '대림2동', '신길1동',
       '신길6동', '신길7동', '여의동', '남영동', '서빙고동', '용산2가동', '원효로2동', '이태원1동',
       '한강로동', '효창동', '상하동', '신갈동', '풍덕천2동', '역삼동', '구산동', '녹번동', '수색동',
       '응암2동', '녹양동', '호원1동', '사직동', '종로1.2.3.4가동', '청운효자동', '동화동', '신당동',
       '장충동', '망우3동', '망우본동', '면목5동', '면목7동', '상봉2동', '신내1동', '금촌2동',
       '문산읍', '금곡동', '백현동', '이매1동', '수진2동', '시흥동', '신촌동', '상대원1동',
       '상대원2동', '마장동', '용답동', '월곡2동', '가락2동', '잠실2동', '율천동', '정왕본동',
       '관양2동', '목3동', '신월2동', '신정2동', '신정6동', '지평면', '능서면', '대림3동',
       '영등포동', '보광동', '이촌1동', '이태원2동', '후암동', '유림동', '송산1동', '신포동', '약수동',
       '중림동', '덕풍3동', '신흥1동', '신흥2동', '상대원3동', '성남동', '사근동', '길음2동',
       '문정2동',

In [8]:
region_doId_mapping = {}
unique_doRegions = merged_df['OB'].unique()
for idx, doRegion in enumerate(unique_doRegions):
    region_doId_mapping[doRegion] = idx

In [9]:
len(region_doId_mapping)

690

In [10]:
# 고유 ID와 지역 매핑 정보를 저장할 데이터프레임 생성
region_doId_df = pd.DataFrame(list(region_doId_mapping.items()), columns=['Region', 'ID'])

# 데이터 타입 추가
region_doId_df['DataType'] = "FILE"

# 랜덤 이름을 추가
users = ['Admin','JH','JY','MS','SY','YS','Gasfard', 'Pierre', 'Santiago','Pel','Royd','Dhino','Charlie']
region_doId_df['User'] = np.random.choice(users, len(region_doId_df))

# 현재 시간을 얻어 'CreatedTime' 열에 추가
region_doId_df['CreatedTime'] = datetime.now()

# Region 값을 가져와 설명을 추가
region_doId_df['description'] = region_doId_df['Region'] + '에 대한 설명'

# 해시태그 키워드
region_doId_df['Keyword'] = [f"[{region}, '지역', '수도권']" for region in region_doId_df['Region']]


region_doId_df = region_doId_df[['ID', 'Region','DataType', 'User', 'CreatedTime', 'description', 'Keyword']]





In [11]:
region_doId_df

,ID,Region,DataType,User,CreatedTime,description,Keyword
0,0,성수2가1동,FILE,Dhino,2023-11-06 15:45:06.917931,성수2가1동에 대한 설명,"[성수2가1동, '지역', '수도권']"
1,1,성수2가3동,FILE,Admin,2023-11-06 15:45:06.917931,성수2가3동에 대한 설명,"[성수2가3동, '지역', '수도권']"
2,2,왕십리도선동,FILE,SY,2023-11-06 15:45:06.917931,왕십리도선동에 대한 설명,"[왕십리도선동, '지역', '수도권']"
3,3,길음1동,FILE,YS,2023-11-06 15:45:06.917931,길음1동에 대한 설명,"[길음1동, '지역', '수도권']"
4,4,삼선동,FILE,JH,2023-11-06 15:45:06.917931,삼선동에 대한 설명,"[삼선동, '지역', '수도권']"
...,...,...,...,...,...,...,...
685,685,암사1동,FILE,YS,2023-11-06 15:45:06.917931,암사1동에 대한 설명,"[암사1동, '지역', '수도권']"
686,686,철산2동,FILE,Charlie,2023-11-06 15:45:06.917931,철산2동에 대한 설명,"[철산2동, '지역', '수도권']"
687,687,광명1동,FILE,Charlie,2023-11-06 15:45:06.917931,광명1동에 대한 설명,"[광명1동, '지역', '수도권']"
688,688,갈산1동,FILE,Santiago,2023-11-06 15:45:06.917931,갈산1동에 대한 설명,"[갈산1동, '지역', '수도권']"


In [13]:
# CSV 파일로 지역과 ID 매핑 정보 저장
region_doId_df.to_csv('ID_DOID_Mapping_V2.csv', index=False)
